In [19]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [20]:
%sendtofile config.txt

wifiname     BV6000
wifipassword bubblino
mqttbroker   192.168.43.1
pinled       2
pinmotorA    13
pinmotorB    12
motordir     -1
motoronms    200
motoroffms   800
boardname    esp8266sens2

Sent 10 lines (190 bytes) to config.txt.


In [9]:
%sendtofile --source ../Sensor_Kennel/BME280_funcs.py

Sent 79 lines (3304 bytes) to BME280_funcs.py.


In [3]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [4]:
%sendtofile main.py

import time
from machine import Pin, I2C
from mqtt_as import config, MQTTClient
import uasyncio as asyncio
from BME280_funcs import bme280init, readBME280
from uasyncio.queues import Queue

fconfig = dict(x.split()  for x in open("config.txt"))
pinled = Pin(int(fconfig["pinled"]), Pin.OUT)
for i in range(10):
    pinled.value(i%2)
    time.sleep_ms(300)
    
i2c = machine.I2C(scl=machine.Pin(5), sda=machine.Pin(4))
print(list(map(hex, i2c.scan())))

config['server'] = fconfig["mqttbroker"]
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]
config['mqttchannel'] = fconfig["boardname"]
boardname = fconfig["boardname"].encode()    

tbme280A = boardname+b'/bme280/A'
tbme280B = boardname+b'/bme280/B'

pinmotorA = Pin(int(fconfig["pinmotorA"]), Pin.OUT)
pinmotorB = Pin(int(fconfig["pinmotorB"]), Pin.OUT)
pinmotorA.value(0)
pinmotorB.value(0)

motordir = int(fconfig["motordir"])
motoronms = int(fconfig["motoronms"])
motoroffms = int(fconfig["motoroffms"])

async def ledtask():
    while True:
        pinled.value(0)
        await asyncio.sleep_ms(20 if client.isconnected() else 1600)
        pinled.value(1)
        await asyncio.sleep_ms(2000)

async def fantask():
    while True:
        if motordir != 0:
            pinmotor = pinmotorA  if motordir == 1  else pinmotorB
            pinmotor.value(1)
            await asyncio.sleep_ms(motoronms)
            pinmotor.value(0)
            await asyncio.sleep_ms(motoroffms + 2000*(1-client.isconnected()))
        else:
            await asyncio.sleep_ms(300)
        
async def bme280task(addr, topic):
    btoinit = True
    while True:
        try:
            if btoinit:
                bme280init(i2c, addr)
                btoinit = False
                await asyncio.sleep_ms(100)
            meas = readBME280(addr)
            payload = " ".join(map(str, meas))
            if client.isconnected():
                print(topic, meas)
                await client.publish(topic, payload)
        except OSError as e:
            print("bme280task", e)
            btoinit = True
        await asyncio.sleep_ms(500)

async def callbackcmdtask(topic, msg):
    print("executing", [msg])
    pinled.value(0)
    try:
        exec(msg)
        await client.publish(boardname+'/reply', "1")
    except Exception as e:
        print(e)
        await client.publish(boardname+'/reply/exception', str(e))
    pinled.value(1)
    return
    
def callbackcmd(topic, msg, retained):
    aloop.create_task(callbackcmdtask(topic, msg))
        
async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.subscribe(boardname+"/cmd")
    await client.publish(boardname+'/ip', ipnumber, retain=True)
    
async def mqtttask(bflip=False):
    if bflip and "wifialt" in fconfig:
        assid, awifi_pw, aserver = fconfig["wifialt"].split(",")
        if client._ssid == assid:
            assid, awifi_pw, aserver = fconfig["wifiname"], fconfig["wifipassword"], fconfig["mqttbroker"]
        client._ssid, client._wifi_pw, client.server = assid, awifi_pw, aserver
    try:
        print("connecting to:", client._ssid)
        await client.connect()
        return
    except OSError as e:
        print("client connect", [e])
    aloop.create_task(mqtttask(True))

    
config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
client = MQTTClient(config)
    
aloop = asyncio.get_event_loop()
aloop.create_task(bme280task(0x76, boardname+b'/bme280/A'))
aloop.create_task(bme280task(0x77, boardname+b'/bme280/B'))
aloop.create_task(ledtask())
aloop.create_task(mqtttask())
aloop.create_task(fantask())
aloop.run_forever()



Sent 118 lines (3693 bytes) to main.py.


In [22]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [20]:
# below is no-sensor running to find why it keeps timing out

In [4]:
%sendtofile main.py

import time
from machine import Pin
from mqtt_as import config, MQTTClient
import uasyncio as asyncio

fconfig = dict(x.split()  for x in open("config.txt"))
pinled = Pin(int(fconfig["pinled"]), Pin.OUT)
for i in range(10):
    pinled.value(i%2)
    time.sleep_ms(300)
    
config['server'] = fconfig["mqttbroker"]
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]
config['mqttchannel'] = fconfig["boardname"]
boardname = fconfig["boardname"]

async def ledtask():
    while True:
        pinled.value(0)
        await asyncio.sleep_ms(20 if client.isconnected() else 1600)
        pinled.value(1)
        await asyncio.sleep_ms(2000)
        
async def sendnumbertask():
    i = 0
    topic = boardname + "/number"
    while True:
        if client.isconnected():
            print(i)
            await client.publish(topic, str(i))
        await asyncio.sleep_ms(1000)
        i += 1

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.publish(boardname+'/ip', ipnumber, retain=True)
    
async def mqtttask(bflip=False):
    if bflip and "wifialt" in fconfig:
        assid, awifi_pw, aserver = fconfig["wifialt"].split(",")
        if client._ssid == assid:
            assid, awifi_pw, aserver = fconfig["wifiname"], fconfig["wifipassword"], fconfig["mqttbroker"]
        client._ssid, client._wifi_pw, client.server = assid, awifi_pw, aserver
    try:
        print("connecting to:", client._ssid)
        await client.connect()
        return
    except OSError as e:
        print("client connect", [e])
    aloop.create_task(mqtttask(True))

config['connect_coro'] = onconnecttask
client = MQTTClient(config)
    
aloop = asyncio.get_event_loop()
aloop.create_task(sendnumbertask())
aloop.create_task(ledtask())
aloop.create_task(mqtttask())
aloop.run_forever()



Sent 62 lines (1878 bytes) to main.py.
